In [1]:
### INSTALLATION FOR COLAB USAGE
import torch
pytorch_version = f"torch-{torch.__version__}.html"
!pip install torch-geometric
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 16.7 MB/s eta 0:00:00


In [2]:
### PYTHON LIBRARIES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, Linear, CGConv, BatchNorm
from torch_geometric.nn import global_mean_pool, global_add_pool
from torch.nn import Softmax, ReLU
from torchmetrics import MeanSquaredLogError as MSLE, MeanSquaredError as MSE

In [3]:
### FILE LOADING FOR COLAB USAGE

from google.colab import drive
drive.mount('/content/drive/')
folder = "/content/drive/MyDrive/colab_semiconductors_GNN"

!unzip -qo "{folder}/train.zip" -d /
!unzip -qo "{folder}/test.zip" -d /


Mounted at /content/drive/


In [4]:
### MY UTILITIES LIBRARY
import sys
sys.path.append(folder)

from semiconductors_pipe_funcs import *

In [5]:
# from google.colab import files
# uploaded = files.upload()

This notebook implements a pipeline to convert atomic coordinates from DFT output files into graph representations. It uses PyTorch Geometric to build and train a GNN model to predict groung state (GS) energies and band gap (BG) energies of semiconductors. It was written to approach the competition Nomad2018 Predicting Transparent Conductors hosted by Kaggle. It requires the file semiconductors_pipe_funcs.py where the pipelines are stored.

In [6]:
### SET DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device", device)

### SET SEED
SEED = 42
set_seed(SEED)

Using device cpu


In [7]:
### DATA LOADING AD PREPROCESSING

train = pd.read_csv(f"{folder}/train.csv", index_col='id')
#test = pd.read_csv(f"{folder}/test.csv", index_col='id')

## Rename columns, one-hot encode spacegroup, rescale lattice parameters and separate targets (`E`, `Bandgap`)
X_train, y_train = my_pipeline(train)
#X_test, y_test = my_pipeline(test)

In [8]:
### GRAPHS CONSTRUCTION

## It gathers the element information and atomic coordinates from the DFT output files to build the graphs.
## Since the Nomad2018 database contains structures with widely different supercell sizes,
## all structures have been expanded to 240 atoms using the unit lattice vectors,
## so all structures are finite but are treated on the same footing.
## Nodes have 6 features: atomic number, electronegativity, and a 4 one-hot encoding of the element.
## Edges are build between nodes using a maximum interatomic distance d_max (given in Angstroms). Edge weights are set to 1/r**2.
## 12 graph attributes are 3 element concentrations, 3 rescaled lattice vectors and 6 one-hot enoded spacegroup.

d_max = 4.0
data_list = create_datalist(X_train,y_train, d_max=d_max, element_encoding=0, bond_encoding=0)
#test_list = create_datalist(X_test,y_test, d_max=d_max, element_encoding=0, bond_encoding=0, train=False)

YOU MUST USE train=False FOR BUILDING A TEST LIST
creating list of Data objects using d_max=4.0,element_encoding=0,bond_encoding=0


In [9]:
#torch.save(data_list,"data_list_dmax{d_max}.pt")
#torch.save(test_list,"data_list_dmax{d_max}.pt")

In [10]:
# Wrap data in a data loader

data_size = len(data_list)
train_size = int(0.8 * data_size)
test_size = data_size - train_size

train_dataset, test_dataset = random_split(data_list, [train_size, test_size])

data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=False)

#data_loader = DataLoader(data_list, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_list, batch_size = len(test_list), shuffle=False)


In [11]:
num_node_features = data_list[0].num_node_features
graph_attr_size = data_list[0].graph_attr.size(1)
num_edge_features = data_list[0].edge_attr.size(1)
#print(num_node_features,graph_attr_size,num_edge_features)

In [12]:
### MODEL DEFINITION


class CGC_featured(torch.nn.ModuleDict):
    def __init__(self,num_node_features=num_node_features,graph_attr_size=graph_attr_size,
                 num_edge_features=num_edge_features, node_embedding=32, embedding_size=64):
        super(CGC_featured, self).__init__()

        ## NODE LEVEL CONVOLUTIONAL LAYERS
        self.node_encoder = Linear(num_node_features, node_embedding)
        self.bn0 = BatchNorm(node_embedding)
        self.conv0 = CGConv(node_embedding, dim= num_edge_features)
        self.bn1 = BatchNorm(node_embedding)
        self.conv1 = CGConv(node_embedding, dim= num_edge_features)
        self.bn2 = BatchNorm(node_embedding)
        # self.conv2 = CGConv(node_embedding, dim= num_edge_features)
        # self.bn3 = BatchNorm(node_embedding)
        self.node2embed = Linear(node_embedding, embedding_size)

        ## GRAPH FEATURE LAYERS
        self.graph1 = Linear(graph_attr_size, embedding_size)
        self.graph2 = Linear(embedding_size, embedding_size)

        ## MIXED COMMON LAYER
        self.mix = Linear(embedding_size*2, embedding_size*2)

        self.head_dropout = nn.Dropout(0.1)

        # MIXED NODE-GRAPH LEVEL LAYERS
        self.Emix = Linear(embedding_size*2, embedding_size)
        self.Eout = Linear(embedding_size, 1)

        # MIXED NODE-GRAPH LEVEL LAYERS
        self.Egapmix = Linear(embedding_size*2, embedding_size)
        self.Egapout = Linear(embedding_size, 1)


    def forward(self, x, edge_index, edge_attr, graph_attr, batch_index):
        ## NODE LAYERS

        hidden = self.node_encoder(x)
        hidden = self.bn0(hidden)
        hidden = F.relu(hidden)

        hidden = self.conv0(hidden, edge_index, edge_attr = edge_attr)
        hidden = self.bn1(hidden)
        hidden = F.relu(hidden)

        hidden = self.conv1(hidden, edge_index, edge_attr = edge_attr)
        hidden = self.bn2(hidden)
        hidden = F.relu(hidden)

        # hidden = self.conv2(hidden, edge_index, edge_attr = edge_attr)
        # hidden = self.bn3(hidden)
        # hidden = F.relu(hidden)

        hidden = self.node2embed(hidden)

        ## NODES POOLING
        pooled = global_add_pool(hidden, batch_index)

        ## GLOBAL GRAPH LAYERS
        graph_features = self.graph1(graph_attr)
        graph_features = F.relu(graph_features)

        graph_features = self.graph2(graph_features)
        graph_features = F.relu(graph_features)

        ##COMBINE NODE + GRAPH FEATURES
        hidden = torch.cat([pooled, graph_features], dim=1)

        ## COMMON MIXING
        mix = F.relu(self.mix(hidden))

        ## ENERGY HEAD
        Emix = F.relu(self.Emix(mix))
        Emix = self.head_dropout(Emix)
        E = self.Eout(Emix)

        ## GAP HEAD
        Egapmix = F.relu(self.Egapmix(mix))
        Egapmix = self.head_dropout(Egapmix)
        Egap = self.Egapout(Egapmix)

        pred = torch.cat([E, Egap],dim = 1) ### dim=0 is the batch

        return pred


In [13]:
## MODEL, ERROR FUNCTION AND OPTOMIZER FOR GAP ENERGY
set_seed(SEED)

model = CGC_featured(node_embedding=32,embedding_size=64)
print(model)
loss_fn = MSE()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)

## MODEL TO GPU
model = model.to(device)
loss_fn = loss_fn.to(device)

CGC_featured(
  (node_encoder): Linear(4, 32, bias=True)
  (bn0): BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv0): CGConv(32, dim=1)
  (bn1): BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): CGConv(32, dim=1)
  (bn2): BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (node2embed): Linear(32, 64, bias=True)
  (graph1): Linear(12, 64, bias=True)
  (graph2): Linear(64, 64, bias=True)
  (mix): Linear(128, 128, bias=True)
  (head_dropout): Dropout(p=0.1, inplace=False)
  (Emix): Linear(128, 64, bias=True)
  (Eout): Linear(64, 1, bias=True)
  (Egapmix): Linear(128, 64, bias=True)
  (Egapout): Linear(64, 1, bias=True)
)


In [14]:
##### Early stopper suggesteed by GPT

class EarlyStopper:
    def __init__(self, patience=20, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0    # reset counter if improved
        else:
            self.counter += 1

        if self.counter >= self.patience:
            self.early_stop = True

In [15]:
### TRAINING MODEL

early_stopper = EarlyStopper(patience=30, min_delta=1e-5)

def train(data_loader):
    model.train()
    total_loss = 0
    for batch in data_loader:
      ## USE GPU
      batch.to(device)
      ## RESET GRADIENTS
      optimizer.zero_grad()
      # MODEL FEED
      pred = model(batch.x, batch.edge_index, batch.edge_attr, batch.graph_attr, batch.batch)
      ## LOSS AND GRADIENT CALCULATION
      loss = loss_fn(pred, batch.y)
      loss.backward()
      ## UPDATE BY GRADIENT
      optimizer.step()
      total_loss += loss.item() * batch.num_graphs

    return total_loss / len(data_loader.dataset)



def test(data_loader):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            pred = model(batch.x, batch.edge_index, batch.edge_attr,
                         batch.graph_attr, batch.batch)
            loss = loss_fn(pred, batch.y)
            total_loss += loss.item() * batch.num_graphs

    return total_loss / len(data_loader.dataset)



epochs = 301
for epoch in range(epochs):
    train_loss = train(data_loader)

    if epoch % 10 == 0:
        test_loss = test(test_loader)
        print(f"Epoch {epoch} | train loss: {train_loss:.5f} | test loss: {test_loss:.5f}")

    if early_stopper.early_stop:
          print(f"Early stopping at epoch {epoch}")
          break



Epoch 0 | train loss: 2.80125 | test loss: 0.30566
Epoch 10 | train loss: 0.00611 | test loss: 0.00347
Epoch 20 | train loss: 0.00442 | test loss: 0.00215
Epoch 30 | train loss: 0.00324 | test loss: 0.00171
Epoch 40 | train loss: 0.00287 | test loss: 0.00150
Epoch 50 | train loss: 0.00273 | test loss: 0.00156
Epoch 60 | train loss: 0.00258 | test loss: 0.00167
Epoch 70 | train loss: 0.00255 | test loss: 0.00148
Epoch 80 | train loss: 0.00226 | test loss: 0.00129


KeyboardInterrupt: 

In [ ]:
## TESTING MODEL

df = pd.DataFrame()
model.eval()
test_batch = next(iter(test_loader))
with torch.no_grad():
    test_batch.to(device)
    pred = model(test_batch.x.float(), test_batch.edge_index, test_batch.edge_attr, test_batch.graph_attr, test_batch.batch)
    pred = pred.cpu()
    df["E_real"] = test_batch.y[:,0].tolist()
    df["Egap_real"] = test_batch.y[:,1].tolist()
    df["E_pred"] = pred[:,0]
    df["Egap_pred"] = pred[:,1]
    df["Egap_real"] *= 10.0
    df["Egap_pred"] *= 10.0


In [ ]:
##ERROR COMPUTATION FOR KAGGLE COMPETITION

df["E_log_err"] = ( np.log(df["E_pred"]+1) - np.log(df["E_real"]+1)) **2
df["Egap_log_err"] = ( np.log(df["Egap_pred"]+1) - np.log(df["Egap_real"]+1)) **2

my_error = np.sqrt((df["E_log_err"]+df["Egap_log_err"]).sum(axis=0) / len(df))

my_error

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["E_real"], df["E_pred"], label="Formation Energy")
plt.scatter(0.1*df["Egap_real"], 0.1*df["Egap_pred"], label="Bandgap")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.legend()
plt.title("Model Predictions vs. Ground Truth")
plt.show()

In [ ]:
## FILE PREPARATION FOR KAGGLE SUBMISSION

# model.eval()

# test_batch = next(iter(test_loader))
# with torch.no_grad():
#     test_batch = test_batch.to(device)
#     pred = model(test_batch.x.float(), test_batch.edge_index, test_batch.edge_attr, test_batch.graph_attr, test_batch.batch)
#     pred = pred.cpu()
#     dfs = pd.DataFrame()
#     dfs["formation_energy_ev_natom"] = pred[:,0].tolist()
#     dfs["bandgap_energy_ev"] = pred[:,1].tolist()
#     dfs["bandgap_energy_ev"] *= 10

# dfs.insert(loc=0, column='id', value=np.arange(len(dfs))+1)
# dfs.to_csv('submission.csv', header=True, index=False)

# from google.colab import files

# files.download('submission.csv')